In [11]:
import os
import torch
import pandas as pd
from matplotlib import pyplot as plt

from deep_fields import data_path
from deep_fields.data.crypto.dataloaders import CryptoDataLoader
from deep_fields.models.crypto.portfolio import NonparametricStochasticPortfolio


from torch.distributions import Normal, MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch import kernels 
from deep_fields.models.gaussian_processes.gaussian_processes import multivariate_normal, white_noise_kernel

%matplotlib inline

In [2]:
# data loader
crypto_folder = os.path.join(data_path, "raw", "crypto")
data_folder = os.path.join(crypto_folder, "2021-06-02")
kwargs = {"path_to_data": data_folder,
          "batch_size": 29,
          "span": "month"}
data_loader = CryptoDataLoader('cpu', **kwargs)

# portfolio model
model_param = NonparametricStochasticPortfolio.get_parameters()
portfolio = NonparametricStochasticPortfolio(**model_param)
portfolio.initialize_inference(data_loader)

New Model Set For Inference
Loading Model No Inference
Current Coin 1 ethereum
Current Coin 2 tether
Current Coin 3 binancecoin
Current Coin 4 cardano
Current Coin 5 dogecoin
Current Coin 6 ripple
Current Coin 7 polkadot
Current Coin 8 usd-coin
Current Coin 9 uniswap


In [3]:
ignore_steps = 10
prices = data_loader.price_df.values
prices = torch.Tensor(prices)
prices = prices.permute(1,0)
prices = prices[:,ignore_steps:]
prices_diferences = prices[:,1:] - prices[:,:-1]
prices_returns = (prices[:,1:] - prices[:,:-1])/(prices[:,:-1]+1e-9)

In [4]:
prices_returns

tensor([[ 0.0032,  0.0124, -0.0017,  ..., -0.0060,  0.0072, -0.0015],
        [-0.0037,  0.0109, -0.0018,  ..., -0.0037,  0.0166, -0.0038],
        [ 0.0043,  0.0046, -0.0020,  ...,  0.0007,  0.0013, -0.0029],
        ...,
        [-0.0368,  0.0254,  0.0066,  ..., -0.0050,  0.0120,  0.0329],
        [ 0.0110,  0.0010, -0.0002,  ..., -0.0012,  0.0043, -0.0027],
        [-0.0169,  0.0391,  0.0201,  ..., -0.0094,  0.0122, -0.0180]])

In [20]:
covariates_dimension = 1
number_of_points = 10
gp_mean_distribution = Normal(torch.zeros((number_of_points,covariates_dimension)),
                              torch.ones((number_of_points,covariates_dimension)))

In [13]:
kernel_parameters = {"kernel_sigma":0.1,"kernel_lenght_scales":[1.]*covariates_dimension}

kernel = ScaleKernel(RBFKernel(ard_num_dims=covariates_dimension, requires_grad=True),
                     requires_grad=True) + white_noise_kernel()

kernel_hypers = {"raw_outputscale": torch.tensor(kernel_parameters.get("kernel_sigma")),
                 "base_kernel.raw_lengthscale": torch.tensor(kernel_parameters.get("kernel_lenght_scales"))}

kernel.kernels[0].initialize(**kernel_hypers)
kernel_eval = lambda locations: kernel(locations, locations).evaluate().float()

In [16]:
points = gp_mean_distribution.sample()

In [17]:
points

tensor([ 1.0547,  0.8492,  1.7503,  0.4494,  0.4876, -0.7815, -2.7371, -0.2598,
        -0.2135, -0.3919])

In [19]:
kernel_eval(points).shape

torch.Size([10, 10])

In [31]:
boundaries = torch.tensor([1, 3, 5, 7, 9])

v = torch.tensor([3.2, 6.1, 9.5])


In [32]:
torch.bucketize(v, boundaries)

tensor([2, 3, 5])

In [33]:
prices[:,0]

tensor([4.6751e+04, 1.4811e+03, 9.9831e-01, 2.2094e+02, 1.0536e+00, 4.8587e-02,
        4.6086e-01, 3.3912e+01, 9.9389e-01, 2.6839e+01])